# Trying Cycle Gan with Pytorch

## Imports

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision.models as models

In [ ]:
class Generator(nn.Module):
    def __init__(self, sz_latent, sz_hidden):
        super(Generator, self).__init__()
        self.conv_1 = nn.Conv2d(in_channels=1, out_channels=64, kernel_size=7, stride=(1, 1))
        self.conv_2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=(2, 2))
        self.conv_3 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=(2, 2))
        self.layer_4 = models.resnet18()
        self.layer_5 = models.resnet18()  # 10 times..?!
        self.conv_m_3 = nn.Conv2d(in_channels=256, out_channels=128, kernel_size=3, stride=(2, 2))
        self.conv_m_2 = nn.Conv2d(in_channels=128, out_channels=64, kernel_size=3, stride=(2, 2))
        self.conv_m_1 = nn.Conv2d(in_channels=64, out_channels=1, kernel_size=7, stride=(1, 1))

    def forward(self, x):
        x = F.relu(self.conv_1(x))
        x = F.relu(self.conv_2(x))
        x = F.relu(self.conv_3(x))
        x = F.relu(self.layer_4(x))
        x = F.relu(self.layer_5(x))
        x = F.relu(self.conv_m_3(x))
        x = F.relu(self.conv_m_2(x))
        x = F.sigmoid(self.conv_m_1(x))
        return x

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, sz):
        super(Discriminator, self).__init__()
        self.conv_1 = nn.Conv2d(in_channels=1, out_channels=64, kernel_size=4, stride=(2, 2))
        self.conv_2 = nn.Conv2d(in_channels=64, out_channels=256, kernel_size=4, stride=(2, 2))
        self.conv_3 = nn.Conv2d(in_channels=256, out_channels=1, kernel_size=1, stride=(1, 1))
        # self.fout = nn.Linear(int(sz / 4), 1)  # output size==1 : raw score

    def forward(self, x):
        x = F.relu(self.conv_1(x)) # try leaky relu
        x = F.relu(self.conv_2(x)) # try leaky relu
        x = self.conv_3(x) # try leaky relu
        return x